In [ ]:
from fewshot.data_provider.dataset import Dataset
from fewshot.backbones import ConvNet

from fewshot.algorithms.backbone_pretrain import build_one_layer_classifier, cross_entropy_train
from fewshot.algorithms.fewshot_models import BaselineFewShotModel

import numpy as np
import tqdm
import scipy.stats as st

from IPython.display import clear_output

### Init

Set up all input data and params

In [2]:
dataset_dir = "../../../data/mini_imagenet/"
dataset = Dataset(dataset_dir=dataset_dir)

In [74]:
# common
seed = 11

img_width = 84
img_height = 84
img_depth = 3

# backbone training
backbone_classes = 80
backbone_training_batch_size = 64
backbone_training_epochs = 30
backbone_train_size = 0.9

# fewshot training and testing
n_way = 5
k_shot = 5
training_batch_size = 4
training_batches_per_episode = 100
query_samples_per_class = 16
n_episodes = 50

### Backbone
Create backbone dataset (80 classes), split to train and validattion

In [11]:
backbone_dataset, fewshot_dataset = dataset.split_by_classes(train_size=backbone_classes,
                                                             random_state=seed)

Split by classes with train size = 80 (seed = 11)
Train classes: 80
Test classes: 20
Train data: 48000 samples
Test data:  12000 samples


In [12]:
backbone_train_dataset, backbone_val_dataset = backbone_dataset.split_by_objects(train_size=backbone_train_size,
                                                                                 random_state=seed)

Split by objects with train size = 0.9 (seed = 11)
Train data: 43200 samples
Test data:  4800 samples


Create classifier

In [13]:
backbone = ConvNet(input_size=(img_width, img_height, img_depth))
backbone_classifier = build_one_layer_classifier(backbone,
                                                 backbone_train_dataset.n_classes)

Load weights if we have presaved

In [64]:
#backbone_classifier.load_weights("../../../data/few-models/bb_clf_30.chk")

Train

In [69]:
backbone.set_trainable(True)

In [75]:
cross_entropy_train(
    backbone_classifier,
    backbone_train_dataset.get_batch_generator(batch_size=backbone_training_batch_size,
                                               target_size=(img_width, img_height),
                                               shuffle=True),
    validation_dataset=backbone_val_dataset.get_batch_generator(batch_size=backbone_training_batch_size,
                                                                target_size=(img_width, img_height),
                                                                shuffle=False),
    n_epochs=backbone_training_epochs
)

675/675 [==============================] - 530s 785ms/step - loss: 0.2318 - acc: 0.9281 - val_loss: 4.2936 - val_acc: 0.3877


Save weights if we want in future to continue to train backbone 

In [80]:
#backbone_classifier.save_weights("../../../data/few-models/bb_clf_30.chk")

### Few Shot
Train fewshot model

In [77]:
accs = []
for episode in tqdm.tqdm_notebook(range(n_episodes)):
    fewshot_model = BaselineFewShotModel(backbone, n_way)
    
    # not very simple way to prepare eposide support and query sets
    # FIXME: in future replace it with @bobbythehiver .few_shot_episode_generator() implementation
    # and @schoooler tester version
    episode_dataset, _ = fewshot_dataset.split_by_classes(train_size=n_way, random_state=episode)
    
    # subset support and query datasets
    episode_support, episode_left = episode_dataset.split_by_objects(train_size=k_shot, random_state=episode)
    episode_query, _ = episode_left.split_by_objects(train_size=query_samples_per_class, random_state=episode)
    
    
    fewshot_model.fit(episode_support.get_batch_generator(batch_size=training_batch_size,
                                                          target_size=(img_width, img_height),
                                                          shuffle=True),
                      n_epochs=training_batches_per_episode)
    out = fewshot_model.predict(episode_query.get_batch_generator(batch_size=training_batch_size,
                                                                  target_size=(img_width, img_height),
                                                                  shuffle=False))
    classes = np.array(episode_query.classes)
    acc = np.mean(classes[np.argmax(out, axis=1)] == episode_query.dataframe["class"].values)
    
    accs.append(acc)
    clear_output()
    
    print("Mean accuracy (by {} episodes): {:.2f}%".format(episode + 1, np.mean(accs) * 100))

Mean accuracy (by 50 episodes): 55.52%


### Report

Calc 95% confidence interval and report

In [78]:
acc_int = st.t.interval(0.95, len(accs) - 1, loc=np.mean(accs), scale=st.sem(accs))

In [79]:
print("5-shot test metric: {}% +- {}%".format(round(np.mean(accs) * 100, 2),
                                              round((np.mean(acc_int) - acc_int[0]) * 100, 2)))

5-shot test metric: 55.52% +- 2.51%
